# Hedge Engine - Signal-to-Hedge Sizing Tool

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Gregory-307/hedge-engine/blob/main/notebooks/hedge_engine_colab.ipynb)

A general-purpose **signal → hedge sizing** engine for quants.

**What it does**: Takes any normalized score (0-1) and liquidity depth, returns a hedge percentage via monotonic spline interpolation.

**Use cases**:
- Sentiment signals → hedge sizing
- Volatility regime detection → position scaling
- Momentum indicators → risk adjustment
- Any custom signal → configurable response curve

## Setup

Install the package (skip if running locally with package installed):

In [ ]:
# Install from GitHub (uncomment in Colab)
# !pip install -q git+https://github.com/Gregory-307/hedge-engine.git

# Or clone and install locally
# !git clone https://github.com/Gregory-307/hedge-engine.git
# !pip install -e ./hedge-engine

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Dict, Any
from dataclasses import dataclass
import math

## Core Concept: Monotonic Spline Mapping

The hedge engine uses **PCHIP (Piecewise Cubic Hermite Interpolating Polynomial)** to create a smooth, monotonic curve from configurable knots.

Why monotonic? A higher risk signal should **never** produce a lower hedge - that would be counterintuitive and dangerous.

In [ ]:
from scipy.interpolate import PchipInterpolator

# Define curve knots: (score, hedge_pct)
# These are configurable per signal type
DEFAULT_KNOTS = [
    (0.0, 0.05),   # Very low risk signal -> 5% hedge (always maintain some protection)
    (0.3, 0.25),   # Low-moderate signal -> 25% hedge
    (0.7, 0.75),   # High signal -> 75% hedge  
    (1.0, 1.00),   # Maximum signal -> 100% hedge
]

def build_spline(knots: List[Tuple[float, float]]) -> PchipInterpolator:
    """Build monotonic spline from (score, hedge) knots."""
    scores = np.array([k[0] for k in knots])
    hedges = np.array([k[1] for k in knots])
    return PchipInterpolator(scores, hedges, extrapolate=False)

spline = build_spline(DEFAULT_KNOTS)

# Visualize the curve
x = np.linspace(0, 1, 100)
y = spline(x)

plt.figure(figsize=(10, 6))
plt.plot(x, y * 100, 'b-', linewidth=2, label='Hedge Curve')
plt.scatter([k[0] for k in DEFAULT_KNOTS], [k[1] * 100 for k in DEFAULT_KNOTS], 
            color='red', s=100, zorder=5, label='Configurable Knots')
plt.xlabel('Signal Score (0 = low risk, 1 = high risk)', fontsize=12)
plt.ylabel('Hedge Percentage (%)', fontsize=12)
plt.title('Monotonic Spline: Signal -> Hedge Mapping', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(0, 1)
plt.ylim(0, 105)
plt.show()

## Liquidity Weighting

The hedge percentage is weighted by **order book liquidity**. Why?

- Deep liquidity -> can execute large hedges without slippage -> full signal response
- Thin liquidity -> large hedges cause slippage -> dampen signal response

Formula: `weight = log(1 + depth) / log(1 + max_depth)`

Log scale because liquidity impact is diminishing - going from $1M to $2M depth matters more than $9M to $10M.

In [ ]:
MAX_DEPTH = 10_000_000  # $10M reference depth
LOG_MAX_DEPTH = math.log1p(MAX_DEPTH)

def liquidity_weight(depth_usd: float) -> float:
    """Weight signal by order book depth (0-1 scale)."""
    return min(1.0, math.log1p(depth_usd) / LOG_MAX_DEPTH)

# Visualize liquidity weighting
depths = np.logspace(4, 8, 100)  # $10K to $100M
weights = [liquidity_weight(d) for d in depths]

plt.figure(figsize=(10, 6))
plt.semilogx(depths / 1e6, weights, 'g-', linewidth=2)
plt.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Max weight (1.0)')
plt.axvline(x=10, color='orange', linestyle='--', alpha=0.5, label='Reference depth ($10M)')
plt.xlabel('Order Book Depth ($ millions)', fontsize=12)
plt.ylabel('Liquidity Weight', fontsize=12)
plt.title('Liquidity Weighting: Deeper Books -> Stronger Signal Response', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print("Example weights:")
for depth in [100_000, 500_000, 1_000_000, 5_000_000, 10_000_000]:
    print(f"  ${depth/1e6:.1f}M depth -> weight = {liquidity_weight(depth):.3f}")

## The Core Function: `compute_hedge`

Combines signal score + liquidity weighting -> hedge percentage

In [ ]:
def compute_hedge(
    score: float,
    depth_usd: float,
    spline: PchipInterpolator,
    max_hedge_pct: float = 1.0
) -> Tuple[float, float]:
    """
    Compute hedge percentage from signal score and liquidity.
    
    Args:
        score: Normalized signal (0-1, where 1 = max risk)
        depth_usd: Order book depth at +/-1% from mid price
        spline: Monotonic interpolator for score -> hedge mapping
        max_hedge_pct: Maximum allowed hedge (default 100%)
    
    Returns:
        (hedge_pct, confidence) tuple
    """
    # Apply liquidity weighting
    weight = liquidity_weight(depth_usd)
    effective_score = score * weight
    
    # Evaluate spline
    hedge_pct = float(spline(effective_score))
    hedge_pct = max(0.0, min(max_hedge_pct, hedge_pct))
    
    # Confidence based on liquidity
    confidence = weight
    
    return hedge_pct, confidence

# Test it
spline = build_spline(DEFAULT_KNOTS)

print("Signal Score x Liquidity -> Hedge Sizing")
print("=" * 50)
for score in [0.2, 0.5, 0.8, 1.0]:
    for depth in [1_000_000, 5_000_000, 10_000_000]:
        hedge, conf = compute_hedge(score, depth, spline)
        print(f"Score={score:.1f}, Depth=${depth/1e6:.0f}M -> Hedge={hedge*100:.1f}%, Confidence={conf:.2f}")

## Example Signal Types

The engine is **signal-agnostic**. Here are example normalizers for different signal sources:

In [ ]:
@dataclass
class SignalConfig:
    """Configuration for a signal type."""
    name: str
    knots: List[Tuple[float, float]]
    description: str

# Different curves for different signal types
SIGNAL_CONFIGS = {
    "sentiment": SignalConfig(
        name="Sentiment",
        knots=[(0.0, 0.05), (0.3, 0.25), (0.7, 0.75), (1.0, 1.0)],
        description="Social media / news sentiment (0=bullish, 1=bearish)"
    ),
    "volatility": SignalConfig(
        name="Volatility Regime",
        knots=[(0.0, 0.10), (0.4, 0.30), (0.8, 0.80), (1.0, 1.0)],
        description="Realized vol percentile (0=calm, 1=extreme)"
    ),
    "momentum": SignalConfig(
        name="Momentum Reversal",
        knots=[(0.0, 0.0), (0.5, 0.20), (0.9, 0.60), (1.0, 0.90)],
        description="Mean reversion signal (0=trending, 1=overextended)"
    ),
    "funding": SignalConfig(
        name="Funding Rate",
        knots=[(0.0, 0.05), (0.3, 0.15), (0.6, 0.50), (1.0, 1.0)],
        description="Perp funding extremity (0=neutral, 1=extreme long crowding)"
    ),
}

# Visualize all signal curves
plt.figure(figsize=(12, 8))
x = np.linspace(0, 1, 100)

for name, config in SIGNAL_CONFIGS.items():
    spline = build_spline(config.knots)
    y = spline(x)
    plt.plot(x, y * 100, linewidth=2, label=f"{config.name}")

plt.xlabel('Normalized Signal Score', fontsize=12)
plt.ylabel('Hedge Percentage (%)', fontsize=12)
plt.title('Signal-Specific Hedge Curves (Configurable per Use Case)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(loc='upper left')
plt.xlim(0, 1)
plt.ylim(0, 105)
plt.show()

print("\nSignal Descriptions:")
for name, config in SIGNAL_CONFIGS.items():
    print(f"  {config.name}: {config.description}")

## Example: Normalizing Raw Signals

Real signals need normalization to the 0-1 range. Here are example normalizers:

In [ ]:
def normalize_sentiment(raw_score: float, min_val: float = -1.0, max_val: float = 1.0) -> float:
    """
    Normalize sentiment from [-1, 1] (bullish to bearish) to [0, 1].
    
    Raw: -1 = very bullish, +1 = very bearish
    Normalized: 0 = bullish (low hedge), 1 = bearish (high hedge)
    """
    return (raw_score - min_val) / (max_val - min_val)

def normalize_volatility(realized_vol: float, lookback_vols: List[float]) -> float:
    """
    Normalize volatility to percentile rank.
    
    Returns: 0 = lowest vol in lookback, 1 = highest vol
    """
    sorted_vols = sorted(lookback_vols)
    rank = sum(1 for v in sorted_vols if v <= realized_vol)
    return rank / len(sorted_vols)

def normalize_funding(funding_rate: float, extreme_threshold: float = 0.001) -> float:
    """
    Normalize funding rate to 0-1.
    
    Raw: -0.001 to +0.001 typical, extremes beyond
    Normalized: 0 = neutral, 1 = extreme long crowding
    """
    # Clip and normalize
    normalized = (funding_rate + extreme_threshold) / (2 * extreme_threshold)
    return max(0.0, min(1.0, normalized))

# Demo
print("Sentiment Normalization:")
for raw in [-0.8, -0.3, 0.0, 0.5, 0.9]:
    norm = normalize_sentiment(raw)
    print(f"  Raw={raw:+.1f} -> Normalized={norm:.2f}")

print("\nFunding Rate Normalization:")
for rate in [-0.0005, 0.0, 0.0003, 0.0008, 0.0015]:
    norm = normalize_funding(rate)
    print(f"  Rate={rate:+.4f} -> Normalized={norm:.2f}")

## Full Example: Multi-Signal Hedge Decision

Combine multiple signals into a single hedge decision:

In [ ]:
def compute_multi_signal_hedge(
    signals: Dict[str, float],  # signal_name -> raw_value
    weights: Dict[str, float],  # signal_name -> weight (must sum to 1)
    depth_usd: float,
    signal_configs: Dict[str, SignalConfig] = SIGNAL_CONFIGS
) -> Dict[str, Any]:
    """
    Compute hedge from multiple weighted signals.
    
    Returns detailed breakdown of each signal's contribution.
    """
    results = {
        "signals": {},
        "weighted_hedge": 0.0,
        "confidence": 0.0,
        "depth_usd": depth_usd,
    }
    
    total_hedge = 0.0
    total_conf = 0.0
    
    for sig_name, raw_value in signals.items():
        if sig_name not in signal_configs:
            continue
            
        config = signal_configs[sig_name]
        weight = weights.get(sig_name, 0.0)
        
        # Build spline for this signal type
        spline = build_spline(config.knots)
        
        # Assume raw_value is already normalized 0-1 for this demo
        hedge_pct, conf = compute_hedge(raw_value, depth_usd, spline)
        
        results["signals"][sig_name] = {
            "raw_value": raw_value,
            "hedge_pct": hedge_pct,
            "weight": weight,
            "contribution": hedge_pct * weight,
        }
        
        total_hedge += hedge_pct * weight
        total_conf += conf * weight
    
    results["weighted_hedge"] = total_hedge
    results["confidence"] = total_conf
    
    return results

# Example: Combine sentiment + volatility + funding
signals = {
    "sentiment": 0.7,    # Somewhat bearish
    "volatility": 0.85,  # High vol regime
    "funding": 0.6,      # Elevated long crowding
}

weights = {
    "sentiment": 0.4,
    "volatility": 0.35,
    "funding": 0.25,
}

result = compute_multi_signal_hedge(signals, weights, depth_usd=5_000_000)

print("Multi-Signal Hedge Decision")
print("=" * 50)
print(f"Order Book Depth: ${result['depth_usd']/1e6:.1f}M")
print()
print("Individual Signals:")
for name, data in result["signals"].items():
    print(f"  {name.capitalize():12} | Score={data['raw_value']:.2f} | "
          f"Hedge={data['hedge_pct']*100:5.1f}% | "
          f"Weight={data['weight']:.2f} | "
          f"Contribution={data['contribution']*100:5.1f}%")
print()
print(f"Final Weighted Hedge: {result['weighted_hedge']*100:.1f}%")
print(f"Confidence: {result['confidence']:.2f}")

## Using the REST API

If running the full service with Docker:

In [ ]:
# Uncomment to test against running service
# import httpx
# 
# API_URL = "http://localhost:8000"
# 
# # Health check
# resp = httpx.get(f"{API_URL}/healthz")
# print(f"Health: {resp.json()}")
# 
# # Get hedge recommendation
# resp = httpx.post(f"{API_URL}/hedge", json={
#     "asset": "BTC",
#     "amount_usd": 100_000,
#     "override_score": 0.7
# })
# print(f"Hedge: {resp.json()}")
# 
# # Prometheus metrics
# resp = httpx.get(f"{API_URL}/metrics")
# print(f"Metrics (first 500 chars):\n{resp.text[:500]}")

## Performance Characteristics

The core `compute_hedge` function is designed for **sub-50 microsecond latency**:

In [ ]:
import time

spline = build_spline(DEFAULT_KNOTS)

# Warm up
for _ in range(100):
    compute_hedge(0.5, 5_000_000, spline)

# Benchmark
iterations = 10_000
start = time.perf_counter()
for i in range(iterations):
    score = (i % 100) / 100
    compute_hedge(score, 5_000_000, spline)
elapsed = time.perf_counter() - start

avg_us = (elapsed / iterations) * 1_000_000
print(f"Benchmark: {iterations:,} iterations")
print(f"Total time: {elapsed*1000:.2f} ms")
print(f"Average per call: {avg_us:.2f} us")
print(f"Throughput: {iterations/elapsed:,.0f} calls/second")

## Summary

**Hedge Engine** is a general-purpose tool for quants:

1. **Signal-agnostic**: Works with any normalized 0-1 score
2. **Configurable curves**: Different spline knots for different signal types
3. **Liquidity-aware**: Dampens response in thin markets
4. **Multi-signal ready**: Combine weighted signals into single decision
5. **Low-latency**: Sub-50us compute, suitable for real-time trading
6. **Production-ready**: Circuit breaker, audit logging, Prometheus metrics

### Next Steps

- Define your signal types and normalization logic
- Configure spline knots based on backtesting
- Integrate with your data pipeline
- Run walk-forward validation before live deployment